In [1]:
import os
import numpy as np
import pandas as pd
from scipy.spatial import distance
import matplotlib.pyplot as plt
import pickle
from math import inf

In [2]:
age_to_be_analysed = 56 # change to 28 if you need to

if age_to_be_analysed == 28:
    data_folder = "//storage.corp.brain.mpg.de/data/Projects/Fritz_Lukas/trex_videos/data/1mpf"
elif age_to_be_analysed == 56:
    data_folder = "//storage.corp.brain.mpg.de/data/Projects/Fritz_Lukas/trex_videos/data/2mpf"

file_list = pd.Series(os.listdir(data_folder))
file_list.columns =  ["files"]

# filter for tracking data (statistics folder not relevant for now)
fish_id = file_list.str.contains("fish")
file_list_filtered = file_list.loc[fish_id]

# identify genotypes and number of videos for each, save as dictionary.
string_split_list = pd.DataFrame(file_list_filtered.str.split("_"))
string_split_list.columns = ["split_string"]
id_df = pd.DataFrame(string_split_list.split_string.tolist(), columns = ['age', 'genotype', 'video', 'fish'])
genotypes = id_df['genotype'].unique()
for genotype in genotypes:
    genotype_videos = []
    videos = id_df.loc[id_df['genotype'] == genotype]['video'].unique()
    if 'data_dict' not in locals():
        data_dict = {genotype: videos.tolist()}
        fish = id_df.loc[id_df['genotype'] == genotype]['fish'].unique().tolist()
    else:
        data_dict.update({genotype: videos.tolist()})
        
data_folder = data_folder + "/" + id_df['age'][0]

In [3]:
# change the value below to 1 if you want to get the pictures with the individual trajectories
plot_trajectories = 0

In [4]:
range_a = range(0,53970,30)
range_b = range(30,54000,30)

for genotype in genotypes: 
    videos2analyse = data_dict[genotype]
    for video in videos2analyse:
        print('Analysis of: ' + str(genotype) + ', video ' + str(video))
        trajectories = np.empty(54000*20*2).reshape(54000,20,2)
        bodylengths = np.empty(20)
        velocity = np.empty(54000*20).reshape(54000,20)
        fish_id = 0
        for fishID in fish:
            file2load = data_folder + '_' + str(genotype) + '_' + str(video) + '_' + fishID 

            fish_df = np.load(file2load)
            bodylengths[fish_id] = np.median(fish_df['midline_length'])
            df_length = (np.shape(fish_df['X']))[0]
            if df_length < 54000:
                N = 54000 - df_length
                x = np.append(fish_df['X'], np.repeat(np.nan, N))
                y = np.append(fish_df['Y'], np.repeat(np.nan, N))
                trajectories[:,fish_id,0] = x
                trajectories[:,fish_id,1] = y
                #velocity[:,fish_id] = np.append(fish_df['SPEED']/bodylengths[fish_id], np.repeat(np.nan, N))
                velocity[:,fish_id] = np.append(fish_df['SPEED'] , np.repeat(np.nan, N))
            elif df_length > 54000:
                trajectories[:,fish_id,0] = fish_df['X'][0:54000]
                trajectories[:,fish_id,1] = fish_df['Y'][0:54000]
                #velocity[:,fish_id] = fish_df['SPEED'][0:54000]/bodylengths[fish_id]  
                velocity[:,fish_id] = fish_df['SPEED'][0:54000]
            else:
                trajectories[:,fish_id,0] = fish_df['X']
                trajectories[:,fish_id,1] = fish_df['Y']
                #velocity[:,fish_id] = fish_df['SPEED']/bodylengths[fish_id]
                velocity[:,fish_id] = fish_df['SPEED']
            fish_id += 1
        # plot trajectories of all fish 
        if plot_trajectories:
            y = 0
            fig,ax = plt.subplots(figsize=(20,10), nrows=3, ncols=7)
            ax[2, 6].plot(trajectories[:,:,0], trajectories[:,:,1], linewidth=0.01)
            i = 0
            for x in range(3):
                for y in range(7):
                    ax[x,y].plot(trajectories[:,i,0], trajectories[:,i,1], linewidth = 0.1)
                    ax[x,y].axis('off')
                    ax[x,y].set_title('Fish ' + str(i+1), fontsize = 14, fontweight ='bold') 
                    i += 1
                    if i == 20:
                        break
            plt.axis('off') 
            save_string = genotype + '_' + video + '.png'
            plt.savefig(save_string)

        velocity_median = np.nanmedian(velocity, axis = 1)
        # sum up velocity to every second    
        vel_sec_median = [np.nansum(velocity_median[i-30:i]) for i in range(30,54000,30)]
        
        velocity_mean = np.nanmean(velocity, axis = 1)
         # sum up velocity to every second    
        vel_sec_mean = [np.nansum(velocity_mean[i-30:i]) for i in range(30,54000,30)]
        
        # perform analysis of distance between fish and polarity:
        # distance over time
        neighbor_distance = np.empty(1800)
        neighbor_min = np.empty(1800)
        neighbor_max = np.empty(1800)
        a = 0
        for frame in range(0,54000,30): # adapt to every second
            positionMatrix = pd.DataFrame([trajectories[frame,:,0], trajectories[frame,:,1]]).transpose()
            trialMatrix = pd.DataFrame(distance.squareform(distance.pdist(positionMatrix)))
            trialMatrix[trialMatrix==0]=np.nan  # set to nan, otherwise the min will identify the same animal.
            trialMatrix[trialMatrix==inf]=np.nan
            neighbor_distance[a] = np.nanmedian(trialMatrix)
            neighbor_min[a] = np.nanmedian(np.nanmin(trialMatrix))
            neighbor_max[a] = np.nanmedian(np.nanmax(trialMatrix))
            a += 1
       # polarity of animals over time this doesn't work too well, requires more work: not in all frames, the animals are detected, resulting in many bad results
       # dx = trajectories[range_b,:,0] - trajectories[range_a,:,0]
       # dy = trajectories[range_b,:,1] - trajectories[range_a,:,1]
       # v0 = np.nansum(((dx**2 + dy**2)**0.5), axis = 1)/(20 - np.sum(np.isnan(dx), axis = 1)) # average absolute velocity normalized by number of fish
       # rho = (((np.nansum(dx, axis = 1)**2) + (np.nansum(dy, axis = 1)**2))**0.5) /(v0*(20 - np.sum(np.isnan(dx), axis = 1)))

        # save to dictionary:
        if 'analysis_dict' not in locals():
            analysis_dict = {genotype: {video: {'velocity': vel_sec_median, 
                                                'velocity_mean': vel_sec_mean,
                                                'nearestneighbor': neighbor_min, 
                                                'neighbor': neighbor_distance,
                                                'neighbor_max': neighbor_max,
                                                'trajectory': trajectories}}}
        elif genotype not in analysis_dict:
            analysis_dict[genotype] = {video: {'velocity': vel_sec_median, 
                                               'velocity_mean': vel_sec_mean,
                                               'nearestneighbor': neighbor_min, 
                                               'neighbor': neighbor_distance,
                                               'neighbor_max': neighbor_max,
                                               'trajectory': trajectories}}
        else:
            analysis_dict[genotype][video] = {'velocity': vel_sec_median, 
                                              'velocity_mean': vel_sec_mean,
                                              'nearestneighbor': neighbor_min, 
                                              'neighbor': neighbor_distance,
                                              'neighbor_max': neighbor_max,
                                              'trajectory': trajectories}


Analysis of: MT02KO, video 01
Analysis of: MT02KO, video 19
Analysis of: MT02KO, video 20
Analysis of: MT02KO, video 09
Analysis of: MT02KO, video 21


C:\Users\gemmera\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
<ipython-input-4-22cb2c2c7178>:59: RuntimeWarning: Mean of empty slice
  velocity_mean = np.nanmean(velocity, axis = 1)


Analysis of: MT02KO, video 23
Analysis of: MT02KO, video 02
Analysis of: MT02KO, video 03
Analysis of: MT02KO, video 04
Analysis of: MT02KO, video 05
Analysis of: MT02KO, video 06
Analysis of: MT02KO, video 07


C:\Users\gemmera\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
<ipython-input-4-22cb2c2c7178>:59: RuntimeWarning: Mean of empty slice
  velocity_mean = np.nanmean(velocity, axis = 1)


Analysis of: MT02KO, video 08


...

Analysis of: MT02WT, video 10
Analysis of: MT02WT, video 11
Analysis of: MT02WT, video 12
Analysis of: MT02WT, video 13
Analysis of: MT02WT, video 14
Analysis of: MT02WT, video 15
Analysis of: MT02WT, video 16
Analysis of: MT02WT, video 17
Analysis of: MT02WT, video 18
Analysis of: MT02WT, video 19
Analysis of: MT02WT, video 20


C:\Users\gemmera\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
<ipython-input-4-22cb2c2c7178>:59: RuntimeWarning: Mean of empty slice
  velocity_mean = np.nanmean(velocity, axis = 1)


Analysis of: MT02WT, video 21
Analysis of: MT02WT, video 22
Analysis of: MT02WT, video 23
Analysis of: MT02WT, video 25
Analysis of: MT02WT, video 26


C:\Users\gemmera\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
<ipython-input-4-22cb2c2c7178>:59: RuntimeWarning: Mean of empty slice
  velocity_mean = np.nanmean(velocity, axis = 1)


Analysis of: MT02WT, video 29


Save output below

In [5]:
if age_to_be_analysed == 28:
    analysis_df_28dpf = pd.DataFrame.from_dict({(i,j): analysis_dict[i][j] 
                           for i in analysis_dict.keys() 
                           for j in analysis_dict[i].keys()},
                       orient='index')
    analysis_df_28dpf.to_pickle('TRex_analysis_28dpf.pkl')
    output = open('TRex_analysis_28dpf.pkl', 'wb')
    
elif age_to_be_analysed == 56:
    analysis_df_56dpf = pd.DataFrame.from_dict({(i,j): analysis_dict[i][j] 
                           for i in analysis_dict.keys() 
                           for j in analysis_dict[i].keys()},
                       orient='index')
    analysis_df_56dpf.to_pickle('TRex_analysis_56dpf.pkl')
    output = open('TRex_analysis_56dpf.pkl', 'wb')

pickle.dump(analysis_dict, output)
output.close()
